# Group information

# Imports

In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
import random
import optuna

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import HeNormal

pd.set_option('display.max_columns', 500)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [288]:
# Data manager class

class Data_Manager() : 

    def __init__(self) : 
        # Y_train
        self.train_a = pd.DataFrame() 
        self.train_b = pd.DataFrame()
        self.train_c = pd.DataFrame()

        self.X_train_observed_a = pd.DataFrame()
        self.X_train_observed_b = pd.DataFrame()
        self.X_train_observed_c = pd.DataFrame()

        self.X_train_estimated_a = pd.DataFrame()
        self.X_train_estimated_b = pd.DataFrame()
        self.X_train_estimated_c = pd.DataFrame()

        self.X_test_estimated_a = pd.DataFrame()
        self.X_test_estimated_b = pd.DataFrame()
        self.X_test_estimated_c = pd.DataFrame()

        self.data_A = pd.DataFrame()    
        self.data_B = pd.DataFrame()
        self.data_C = pd.DataFrame()

        self.data = pd.DataFrame()
        self.X_test_estimated = pd.DataFrame()

        # X_train_obs, Y_train_obs
        self.data_A_obs = pd.DataFrame()    
        self.data_B_obs = pd.DataFrame()
        self.data_C_obs = pd.DataFrame()
        
        # X_train_obs, Y_train_obs
        self.data_A_es = pd.DataFrame()
        self.data_B_es = pd.DataFrame()
        self.data_C_es = pd.DataFrame()

        self.amplitude = np.zeros(3) # amp_a, amp_b, amp_c

    def data_loader(self): 
        """
        Function that loads the datasets into data manager, loads all data 
        """

        self.train_a = pd.read_parquet('A/train_targets.parquet')
        self.train_a = self.train_a.rename(columns={"time":"date_forecast"})

        self.train_b = pd.read_parquet('B/train_targets.parquet')
        self.train_b = self.train_b.rename(columns={"time":"date_forecast"})

        self.train_c = pd.read_parquet('C/train_targets.parquet')
        self.train_c = self.train_c.rename(columns={"time":"date_forecast"})

        self.X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
        self.X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
        self.X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

        self.X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
        self.X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
        self.X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

        self.X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
        self.X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
        self.X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')
    
    def dms2dm(self, dms):
        self.train_a = dms.data['train_a']
        self.train_b = dms.data['train_b']
        self.train_c = dms.data['train_c']

        self.X_train_estimated_a = dms.data['X_train_estimated_a']
        self.X_train_estimated_b = dms.data['X_train_estimated_b']
        self.X_train_estimated_c = dms.data['X_train_estimated_c']

        self.X_train_observed_a = dms.data['X_train_observed_a']
        self.X_train_observed_b = dms.data['X_train_observed_b']
        self.X_train_observed_c = dms.data['X_train_observed_c']

        self.X_test_estimated_a = dms.data['X_test_estimated_a']
        self.X_test_estimated_b = dms.data['X_test_estimated_b']
        self.X_test_estimated_c = dms.data['X_test_estimated_c']

        self.data_A_obs = dms.data['data_A_obs']
        self.data_B_obs = dms.data['data_B_obs']
        self.data_C_obs = dms.data['data_C_obs']

        self.data_A_es = dms.data['data_A_es']
        self.data_B_es = dms.data['data_B_es']
        self.data_C_es = dms.data['data_C_es']

        self.data_A = dms.data['data_A']   
        self.data_B = dms.data['data_B']
        self.data_C = dms.data['data_C']

        self.amplitude = dms.data['amplitude']

    def drop_feature(self, datasets:list[pd.DataFrame], features:list[str]):
        """
        Takes in list of datasets and removes features from the sets

        Returns list of altered datasets
        """

        altered_sets = []

        for set in datasets: 
            for feature in features:

                set = set.drop(feature, axis=1)

            altered_sets.append(set)

        return altered_sets
    
    def combine_data(self): 
        import pandas as pd
        """
        Combines datasets A, B and C into one set containing all features and pv_measurements. 

        Combine_observed_estimated (bool) determines if you want one single set for A B C or keep the observed and estimated
        data split 

        Warning! Data should have no NaN values or be of same frequency before combining! 
        """
        weather_data_A = pd.concat([self.X_train_observed_a, self.X_train_estimated_a], axis=0, ignore_index=True)
        weather_data_B = pd.concat([self.X_train_observed_b, self.X_train_estimated_b], axis=0, ignore_index=True)
        weather_data_C = pd.concat([self.X_train_observed_c, self.X_train_estimated_c], axis=0, ignore_index=True)

        self.data_A = pd.merge(weather_data_A, self.train_a, how="left", on="date_forecast")
        self.data_B = pd.merge(weather_data_B, self.train_b,  on="date_forecast", how="left")
        self.data_C = pd.merge(weather_data_C, self.train_c, on="date_forecast", how="left")

        if ( self.data_A.columns.__contains__("date_calc") ): 
            self.data_A = self.data_A.drop("date_calc", axis=1)
            self.data_B = self.data_B.drop("date_calc", axis=1)
            self.data_C = self.data_C.drop("date_calc", axis=1)

        if (self.train_a.shape[0] > 35000 ) : 
            
            self.data_A = self.makima_interpolate(self.data_A).dropna()
            self.data_B = self.makima_interpolate(self.data_B).dropna()
            self.data_C = self.makima_interpolate(self.data_C).dropna()
        
        elif (self.train_a.shape[0] < 35000) : 
            self.data_A = self.data_A.dropna()
            self.data_B = self.data_B.dropna()
            self.data_C = self.data_C.dropna()

        if self.data_A.isna().sum().sum() > 0 :
            warnings.warn("Warning! Data should have no NaN values or be of same frequency before combining! Use impute or interpolation on data before combining! This could also come from dates in the combined datasets not overlapping fully.")

        return self.data_A, self.data_B, self.data_C

    def impute_data(self, datasets, advanced_imputer=False):

        """
        imputes data to fill in missing values

        takes in a list of datasets

        returns list of imputed data

        removes all columns consisting entirely of nan 
        """

        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer, SimpleImputer
        from tqdm import tqdm
        imputed_sets = []

        for set in tqdm(datasets): 

            # storing columns to lable after impute, also removing date column as this does not work with impute 
            cols = set.columns 

            if set.columns.__contains__("date_forecast"): 
                dates = set["date_forecast"]
            
            if set.columns.__contains__("date_calc"): 
                set = set.drop("date_calc", axis=1)

            cols = set.columns.delete(0)

            set_wo_date = set.drop("date_forecast", axis=1)

            #imputing (estimating) missing values 
            imp = SimpleImputer(missing_values=np.nan, strategy="mean", add_indicator=False)
            imp.fit(set_wo_date)
            set_wo_date = pd.DataFrame(imp.transform(set_wo_date), columns=imp.get_feature_names_out())
            

            # setting column lables basck
            set = set_wo_date
            
            set["date_forecast"] = dates

            #sorting columns 
            cols = cols.tolist()
            cols.insert(0, "date_forecast")

            #set = set.fillna(0.0)

            imputed_sets.append(set)

        return imputed_sets
    
    def iterative_imputer(self, datasets) :

        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer, SimpleImputer
        from tqdm import tqdm
        imputed_sets = []
        imp = IterativeImputer(random_state=0, missing_values=np.nan, add_indicator=False, imputation_order="ascending", skip_complete=True)

        for set in tqdm(datasets): 

            cols = set.columns 

            if set.columns.__contains__("date_forecast"): 
                dates = set["date_forecast"]
            
            if set.columns.__contains__("date_calc"): 
                set = set.drop("date_calc", axis=1)
            
            cols = set.columns.delete(0)

            set_wo_date = set.drop("date_forecast", axis=1)

            print("getting to imputing")
            #imputing (estimating) missing values 
            imp.fit(set_wo_date)
            
            set_wo_date = pd.DataFrame(imp.transform(set_wo_date), columns=imp.get_feature_names_out())

             # setting column lables basck
            set = set_wo_date
            
            set["date_forecast"] = dates

            # set = set.fillna(0.0)


            #sorting columns 
            cols = cols.tolist()
            cols.insert(0, "date_forecast")

            imputed_sets.append(set)

        return imputed_sets

    def resample_data(self, datasets:[pd.DataFrame], freq:str) : 


        """
        resamples the given dataset into the correct frequency. 
        H : hourly 
        15T : 15min 
        """
        
        corr = []


        for set in datasets: 

            mean_set = set.loc[:, set.columns]

            set_hourly = mean_set.resample(freq, on="date_forecast").mean()

            set_dates = mean_set["date_forecast"].dt.date.unique().tolist()

            set_hourly["date_forecast"] = set_hourly.index

            mean_set_corr = set_hourly[set_hourly["date_forecast"].dt.date.isin(set_dates)]

            mean_set_corr.index = pd.RangeIndex(0, len(mean_set_corr))
        
            corr.append(mean_set_corr)


    
        return corr

    def add_feature(dataset, feature_name, data) :

        added_set = dataset[feature_name] = data

        return added_set
    
    def set_info(self, dataset:pd.DataFrame):

        (dataset.info())

    def plot_feature(self, dataset:pd.DataFrame, featureName:str):
        fig, axs = plt.subplots(1, 1, figsize=(20, 10))
        fig.set_label(featureName)

        plt.scatter(dataset["date_forecast"], dataset[featureName], s=3)

        #dataset[['date_forecast', featureName]].set_index("date_forecast").plot(ax=axs, title=featureName, color='red')
       
    def KNNImputing(self, datasets) :
        from sklearn.impute import KNNImputer
        from tqdm import tqdm

        imputed_sets = []

        for set in tqdm(datasets): 

            # storing columns to lable after impute, also removing date column as this does not work with impute 
            cols = set.columns 

            if set.columns.__contains__("date_forecast"): 
                dates = set["date_forecast"]
            
            if set.columns.__contains__("date_calc"): 
                set = set.drop("date_calc", axis=1)

            cols = set.columns.delete(0)

            set_wo_date = set.drop("date_forecast", axis=1)

            #imputing (estimating) missing values 
            imp = KNNImputer(n_neighbors=40, weights="distance", )
            imp.fit(set_wo_date)
            set_wo_date = pd.DataFrame(imp.transform(set_wo_date), columns=imp.get_feature_names_out())
            

            # setting column lables basck
            set = set_wo_date
            
            set["date_forecast"] = dates

            #sorting columns 
            cols = cols.tolist()
            cols.insert(0, "date_forecast")

            ## set = set.fillna(0.0)

            imputed_sets.append(set)

        return imputed_sets

    def makima_interpolate(self, data: pd.DataFrame) -> pd.DataFrame:

        from scipy.interpolate import Akima1DInterpolator
        # Extract non-missing values and their indices

        dates = data["date_forecast"]

        non_nan_indices = data['pv_measurement'].dropna().index
        non_nan_values = data['pv_measurement'].dropna().values

        # Apply the Modified Akima Interpolation
        akima = Akima1DInterpolator(non_nan_indices, non_nan_values)
        interpolated_values = akima(data.index)

        # Replace the original column with the interpolated values
        data['pv_measurement'] = interpolated_values

        data[data["pv_measurement"]< 0] = 0 ; 

        data["date_forecast"] = dates; 

        return data
    
    def normalize_data(self) : 
        from sklearn import preprocessing

        if (self.data_A.empty) : 
            print( "empty ")
            relevant_sets = [attr for attr in dir(self) if not callable(getattr(self, attr)) and not attr.startswith("__") and not attr.__contains__("data") and not attr == 'amplitude' and not attr.__contains__("normalizing")]

        else: 
            print("not emptu")
            relevant_sets = [attr for attr in dir(self) if not callable(getattr(self, attr)) and not attr.startswith("__") and attr.__contains__("data_") and not attr.__contains__("_es") and not attr.__contains__("obs") or attr.__contains__("_estimated_")]
        self.__setattr__("normalizing_consts", {})

        print(relevant_sets)

        min_max_scaler = preprocessing.MinMaxScaler()
        normalizer = preprocessing.Normalizer()
        

        for att in relevant_sets: 
            set : pd.DataFrame = self.__getattribute__(att)

            cols = set.columns 

            if set.columns.__contains__("date_forecast"): 
                dates = set["date_forecast"]
            
            if set.columns.__contains__("date_calc"): 
                set = set.drop("date_calc", axis=1)

            cols = set.columns.delete(0)

            set_wo_date = set.drop("date_forecast", axis=1)

            x = set_wo_date.values

            x_normalized = min_max_scaler.fit_transform(x)

            self.normalizing_consts[att] = (set_wo_date.min(), np.abs(set_wo_date.max() - set_wo_date.min())) ## storing normalizing consts for later 
            
            normalized_set = pd.DataFrame(x_normalized)

            normalized_set.columns = cols

            

            self.__setattr__(att, normalized_set)
        
    def scaling(self, preds, location:str) : 

        """
        FORMAT OF PREDICTIONS SHOULD BE 1 COLUMN WITH PREDS

        LOCATION: A B or C
        """

        relevant_sets = [attr for attr in dir(self) if not callable(getattr(self, attr)) and not attr.startswith("__") and not attr.__contains__("data") and not attr == 'amplitude' and (attr.__contains__("train_a") or attr.__contains__("train_b") or attr.__contains__("train_c"))]
        if (self.data_A.empty) : 
            print( "empty ")
            relevant_sets = [attr for attr in dir(self) if not callable(getattr(self, attr)) and not attr.startswith("__") and not attr.__contains__("data") and not attr == 'amplitude' and not attr.__contains__("normalizing")]

        else: 
            print("not emptu")
            relevant_sets = [attr for attr in dir(self) if not callable(getattr(self, attr)) and not attr.startswith("__") and attr.__contains__("data_") and not attr.__contains__("es") and not attr.__contains__("obs")]

        
        loc_index = 0

        if location.capitalize() == "A" :

            loc_index = 0

        elif location.capitalize() == "B": 
        
            loc_index = 1

        elif location.capitalize() == "C": 

            loc_index = 2
           
        elif ( location.capitalize() == "D"): 

            A = pd.DataFrame(preds[0:720])
            B = pd.DataFrame(preds[720:2*720])
            C = pd.DataFrame(preds[2*720:])

            min = self.normalizing_consts["data_A"][0][0]
            diff = self.normalizing_consts["data_A"][1][0]

            scaled_A = (A+min) * diff



            min = self.normalizing_consts["data_B"][0][0]
            diff = self.normalizing_consts["data_B"][1][0]

            scaled_B = (B+min) * diff



            min = self.normalizing_consts["data_C"][0][0]
            diff = self.normalizing_consts["data_C"][1][0]

            scaled_C = (C+min) * diff


            print(A.shape)
            scaled_set = pd.concat([scaled_A, scaled_B, scaled_C])

            return scaled_set

        relevant_set = relevant_sets[loc_index]

    
        min = self.normalizing_consts[relevant_set][0][0]
        diff = self.normalizing_consts[relevant_set][1][0]

        scaled_set = (preds + min) * diff

        return scaled_set
    
    def combine_overlap_BC(self): 
        import math
        """
        This function is created for merging B and C to remove the nan values apparent when merging pv_measurement to the weather data
        This is because of the observation that B and C overlap and where one is missing the other fills in. 
        Must run combine data first to create data_A B C
        """

        original_B = self.data_B
        original_C = self.data_C  

        b2c_scaling = original_B["pv_measurement"].max()/original_C["pv_measurement"].max()

        print(b2c_scaling)      

        original_C[original_C.isnull()] = self.data_B
        original_B[original_B.isnull()] = self.data_C

        self.data_C = original_C.dropna()
        self.data_B = original_B.dropna()

    def sorting_columns_inMainSets(self):

        A = self.data_A 
        cols = A.columns.tolist()

        #sorting columns 
        cols.remove("date_forecast")
        cols.remove("pv_measurement")
        cols.insert(0, "pv_measurement")
        cols.insert(0, "date_forecast")

        A = A[cols]
        self.data_A = A

        #------------------------------------------------------------# 

        B = self.data_B
        cols = B.columns.tolist()

        #sorting columns 
        cols.remove("date_forecast")
        cols.remove("pv_measurement")
        cols.insert(0, "pv_measurement")
        cols.insert(0, "date_forecast")

        B = B[cols]
        self.data_B = B 

        #------------------------------------------------------------#

        C = self.data_C

        cols = C.columns.tolist()

        #sorting columns 
        cols.remove("date_forecast")
        cols.remove("pv_measurement")
        cols.insert(0, "pv_measurement")
        cols.insert(0, "date_forecast")

        C = C[cols]
        self.data_C = C

        #------------------------------------------------------------#

        A = self.X_test_estimated_a

        cols = A.columns.tolist()

        #sorting columns 
        cols.remove("date_forecast")
        cols.insert(0, "date_forecast")

        A = A[cols]
        self.X_test_estimated_a = A

        #------------------------------------------------------------#

        B = self.X_test_estimated_b

        cols = B.columns.tolist()

        #sorting columns 
        cols.remove("date_forecast")
        cols.insert(0, "date_forecast")

        B = B[cols]
        self.X_test_estimated_b = B

        #------------------------------------------------------------#

        C = self.X_test_estimated_c

        cols = C.columns.tolist()

        #sorting columns 
        cols.remove("date_forecast")
        cols.insert(0, "date_forecast")

        C = C[cols]
        self.X_test_estimated_c = C

        #------------------------------------------------------------#

        if ( not self.data.empty) : 
            data = self.data

            cols = data.columns.tolist()

            #sorting columns 
            cols.remove("date_forecast")
            cols.remove("pv_measurement")
            cols.insert(0, "pv_measurement")
            cols.insert(0, "date_forecast")

            data = data[cols]
            self.data = data

    def remove_constant_periods(self, period_length, ignore_values=[]) :
        from helpers import find_const_interval

        train_a = self.data_A.reset_index(drop=True)
        train_b = self.data_B.reset_index(drop=True)
        train_c = self.data_C.reset_index(drop=True)

        y_train_a_const_idx, ca = find_const_interval(train_a, 'pv_measurement', period_length, ignore_values)
        print('y_train_a anomalies:',ca)

        y_train_b_const_idx, cb = find_const_interval(train_b, 'pv_measurement', period_length, ignore_values)
        print('y_train_b anomalies:',cb)

        y_train_c_const_idx, cc = find_const_interval(train_c, 'pv_measurement', period_length, ignore_values)
        print('y_train_c anomalies:',cc)


        date_forecast_a_const = train_a.iloc[y_train_a_const_idx]['date_forecast']
        date_forecast_a_const_values = date_forecast_a_const.values
        train_a = train_a[~train_a['date_forecast'].isin(date_forecast_a_const_values)]

        date_forecast_b_const = train_b.iloc[y_train_b_const_idx]['date_forecast']
        date_forecast_b_const_values = date_forecast_b_const.values
        train_b = train_b[~train_b['date_forecast'].isin(date_forecast_b_const_values)]

        date_forecast_c_const = train_c.iloc[y_train_c_const_idx]['date_forecast']
        date_forecast_c_const_values = date_forecast_c_const.values
        train_c = train_c[~train_c['date_forecast'].isin(date_forecast_c_const_values)]

        self.data_A = train_a
        self.data_B = train_b
        self.data_C = train_c
                
    def split_timeseries(self, X, y, num_splits):

        num_rows = X.shape[0]

        split_length = num_rows // num_splits

        all_splits = []

        end_idx = split_length

        all_splits.append([np.arange(0, round(split_length*0.9), 1), np.arange(round(split_length*0.9), split_length, 1)])


        while 2*end_idx < num_rows: 

            all_splits.append([np.arange(round(end_idx), end_idx + round(0.9*end_idx), 1), np.arange(round(end_idx*0.9)+end_idx, 2*end_idx, 1)])

            end_idx += split_length

        print(end_idx + split_length)
        rest = num_rows - end_idx 

        all_splits.append([np.arange(round(2), )])

        print(all_splits) 

    def add_location(self, dataset:pd.DataFrame, location):

        if location == "A": loc = 0 
        if location == "B": loc = 1 
        if location == "C": loc = 2

        index = dataset.index

        loc_column = pd.DataFrame()
        loc_column.index = index
        loc_column["location"] = loc

        dataset["location"] = loc

        return dataset
    
    def add_lag_feature(self, target_attribute, lag):

        lag_attribute = target_attribute + "_lag_" + str(lag)

        self.data_A[lag_attribute] = self.data_A[target_attribute].shift(lag).fillna(0)
        self.data_B[lag_attribute] = self.data_B[target_attribute].shift(lag).fillna(0)
        self.data_C[lag_attribute] = self.data_C[target_attribute].shift(lag).fillna(0)

        self.X_test_estimated_a[lag_attribute] = self.X_test_estimated_a[target_attribute].shift(lag).fillna(0)
        self.X_test_estimated_b[lag_attribute] = self.X_test_estimated_b[target_attribute].shift(lag).fillna(0)
        self.X_test_estimated_c[lag_attribute] = self.X_test_estimated_c[target_attribute].shift(lag).fillna(0)
        
    def combine_all_data(self): 

        relevant_sets_A = [attr for attr in dir(self) if attr.__eq__("data_A") or attr.__eq__("X_test_estimated_a")]
        relevant_sets_B = [attr for attr in dir(self) if attr.__eq__("data_B") or attr.__eq__("X_test_estimated_b")]
        relevant_sets_C = [attr for attr in dir(self) if attr.__eq__("data_C") or attr.__eq__("X_test_estimated_c")]

        print(relevant_sets_A)
        print(relevant_sets_B)
        print(relevant_sets_C)


        for set in relevant_sets_A : 
            self.__setattr__(set, self.add_location(self.__getattribute__(set), "A"))
        for set in relevant_sets_B : 
            self.__setattr__(set, self.add_location(self.__getattribute__(set), "B"))
        for set in relevant_sets_C : 
            self.__setattr__(set, self.add_location(self.__getattribute__(set), "C"))


        self.data = pd.concat([self.data_A, self.data_B, self.data_C], ignore_index=True)
        self.X_test_estimated = pd.concat([self.X_test_estimated_a, self.X_test_estimated_b, self.X_test_estimated_c], ignore_index=True)

    def remove_outliers_z_score(self, data:pd.DataFrame, threshold=3) : 
        
        from scipy import stats

        before = self.data["diffuse_rad:W"][0:3*720]

        dates = None 

        if "date_forecast" in data.columns: 

            dates = data["date_forecast"]
            data = data.drop("date_forecast", axis=1)

        z_scores = stats.zscore(data.astype(float))

        outliers = data[z_scores > threshold]
        #outliers = outliers.drop("index", axis=1)
        outliers = outliers.notna()

        indexx = None

        if (len(outliers) > 1): 

            for outlier in outliers: 
                # print(outliers[outlier])

                # print(data[outlier])

                if (outlier != "index"):

                    index = data[outlier].loc[outliers[outlier] == True]

                    data[outlier] = data[outlier].replace(index.index, np.nan)


                    indexx = index
            

            data["date_forecast"] = dates

            data = self.KNNImputing([data])[0]
            data = data.reset_index(drop=True)
            self.sorting_columns_inMainSets()
            print(self.set_info(self.data))
            
            x = np.arange(0, 3*720, 1)
            fig, axs = plt.subplots(1, 1, figsize=(20, 10))
            plt.plot(x, before, c="blue", label="before")            
            plt.plot(x, data["diffuse_rad:W"][0:3*720], c="orange", label="after") 
            plt.legend()           
            plt.show()
            return data

        else: 
            print("no outliers")
            return data
    
    def resample_categorical(self, dataset:pd.DataFrame, feature:str): 

        "resamples the categorical feature to an hourly basis, returns the categorical column"

        hour_df = dataset[dataset["date_forecast"].dt.hour % 1 == 0][feature]

        return hour_df



In [289]:
# instanciate a new datamanager 
dm = Data_Manager()
dm.data_loader()

# Data preperation

## Removing features

### Features with too many NaN Values

In [290]:
dm.X_train_observed_a = dm.X_train_observed_a.drop("snow_density:kgm3", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("snow_density:kgm3", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("snow_density:kgm3", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("snow_density:kgm3", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("snow_density:kgm3", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("snow_density:kgm3", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("snow_density:kgm3", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("snow_density:kgm3", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("snow_density:kgm3", axis=1)


### Cloud base and Ceiling height

In [291]:
dm.X_train_observed_a = dm.X_train_observed_a.drop("ceiling_height_agl:m", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("ceiling_height_agl:m", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("ceiling_height_agl:m", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("ceiling_height_agl:m", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("ceiling_height_agl:m", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("ceiling_height_agl:m", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("ceiling_height_agl:m", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("ceiling_height_agl:m", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("ceiling_height_agl:m", axis=1)

In [292]:
dm.X_train_observed_a["cloud_base_agl:m"][dm.X_train_observed_a["cloud_base_agl:m"].isna()] = -667
dm.X_train_estimated_a["cloud_base_agl:m"][dm.X_train_estimated_a["cloud_base_agl:m"].isna()] = -667
dm.X_test_estimated_a["cloud_base_agl:m"][dm.X_test_estimated_a["cloud_base_agl:m"].isna()] = -667

dm.X_train_observed_b["cloud_base_agl:m"][dm.X_train_observed_b["cloud_base_agl:m"].isna()] = -667
dm.X_train_estimated_b["cloud_base_agl:m"][dm.X_train_estimated_b["cloud_base_agl:m"].isna()] = -667
dm.X_test_estimated_b["cloud_base_agl:m"][dm.X_test_estimated_b["cloud_base_agl:m"].isna()] = -667

dm.X_train_observed_c["cloud_base_agl:m"][dm.X_train_observed_c["cloud_base_agl:m"].isna()] = -667
dm.X_train_estimated_c["cloud_base_agl:m"][dm.X_train_estimated_c["cloud_base_agl:m"].isna()] = -667
dm.X_test_estimated_c["cloud_base_agl:m"][dm.X_test_estimated_c["cloud_base_agl:m"].isna()] = -667


#dm.plot_feature(dm.X_train_observed_a, "cloud_base_agl:m")

/var/folders/kq/hk1l39ys077bc7f9741ypg800000gn/T/ipykernel_36027/3371666875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm.X_train_observed_a["cloud_base_agl:m"][dm.X_train_observed_a["cloud_base_agl:m"].isna()] = -667
/var/folders/kq/hk1l39ys077bc7f9741ypg800000gn/T/ipykernel_36027/3371666875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm.X_train_estimated_a["cloud_base_agl:m"][dm.X_train_estimated_a["cloud_base_agl:m"].isna()] = -667
/var/folders/kq/hk1l39ys077bc7f9741ypg800000gn/T/ipykernel_36027/3371666875.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [293]:
# set the date_forecast interval
start_date = '2020-03-25 00:00:00'
end_date = '2020-03-27 00:00:00'

# filter the data based on the date_forecast interval
data_filtered = dm.X_train_observed_a[(dm.X_train_observed_a['date_forecast'] >= start_date) & (dm.X_train_observed_a['date_forecast'] <= end_date)]

# plot the cloud_base_agl column
#data_filtered.plot(x='date_forecast', y='cloud_base_agl:m', figsize=(20,10), style='.')


In [294]:
cloud_turning_point = '2020-03-26 00:00:00'

### Date calc 

In [295]:
dm.X_train_estimated_a = dm.X_train_estimated_a.drop("date_calc", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("date_calc", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("date_calc", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("date_calc", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("date_calc", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("date_calc", axis=1)

### Snow

In [296]:
dm.X_train_observed_a = dm.X_train_observed_a.drop("fresh_snow_1h:cm", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("fresh_snow_1h:cm", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("fresh_snow_1h:cm", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("fresh_snow_1h:cm", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("fresh_snow_1h:cm", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("fresh_snow_1h:cm", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("fresh_snow_1h:cm", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("fresh_snow_1h:cm", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("fresh_snow_1h:cm", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("fresh_snow_3h:cm", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("fresh_snow_3h:cm", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("fresh_snow_3h:cm", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("fresh_snow_3h:cm", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("fresh_snow_3h:cm", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("fresh_snow_3h:cm", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("fresh_snow_3h:cm", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("fresh_snow_3h:cm", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("fresh_snow_3h:cm", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("fresh_snow_6h:cm", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("fresh_snow_6h:cm", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("fresh_snow_6h:cm", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("fresh_snow_6h:cm", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("fresh_snow_6h:cm", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("fresh_snow_6h:cm", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("fresh_snow_6h:cm", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("fresh_snow_6h:cm", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("fresh_snow_6h:cm", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("snow_depth:cm", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("snow_depth:cm", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("snow_depth:cm", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("snow_depth:cm", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("snow_depth:cm", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("snow_depth:cm", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("snow_depth:cm", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("snow_depth:cm", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("snow_depth:cm", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("snow_drift:idx", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("snow_drift:idx", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("snow_drift:idx", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("snow_drift:idx", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("snow_drift:idx", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("snow_drift:idx", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("snow_drift:idx", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("snow_drift:idx", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("snow_drift:idx", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("snow_melt_10min:mm", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("snow_melt_10min:mm", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("snow_melt_10min:mm", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("snow_melt_10min:mm", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("snow_melt_10min:mm", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("snow_melt_10min:mm", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("snow_melt_10min:mm", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("snow_melt_10min:mm", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("snow_melt_10min:mm", axis=1)

### Wind speed

In [297]:

dm.X_train_observed_a = dm.X_train_observed_a.drop("wind_speed_w_1000hPa:ms", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("wind_speed_w_1000hPa:ms", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("wind_speed_w_1000hPa:ms", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("wind_speed_w_1000hPa:ms", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("wind_speed_w_1000hPa:ms", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("wind_speed_w_1000hPa:ms", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("wind_speed_w_1000hPa:ms", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("wind_speed_w_1000hPa:ms", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("wind_speed_w_1000hPa:ms", axis=1)



### Dew or rime

In [298]:
"""
dm.X_train_observed_a = dm.X_train_observed_a.drop("dew_or_rime:idx", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("dew_or_rime:idx", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("dew_or_rime:idx", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("dew_or_rime:idx", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("dew_or_rime:idx", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("dew_or_rime:idx", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("dew_or_rime:idx", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("dew_or_rime:idx", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("dew_or_rime:idx", axis=1)

dm.X_train_observed_a = dm.X_train_observed_a.drop("prob_rime:p", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("prob_rime:p", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("prob_rime:p", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("prob_rime:p", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("prob_rime:p", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("prob_rime:p", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("prob_rime:p", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("prob_rime:p", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("prob_rime:p", axis=1)
"""


'\ndm.X_train_observed_a = dm.X_train_observed_a.drop("dew_or_rime:idx", axis=1)\ndm.X_train_observed_b = dm.X_train_observed_b.drop("dew_or_rime:idx", axis=1) \ndm.X_train_observed_c = dm.X_train_observed_c.drop("dew_or_rime:idx", axis=1) \n\ndm.X_train_estimated_a = dm.X_train_estimated_a.drop("dew_or_rime:idx", axis=1)\ndm.X_train_estimated_b = dm.X_train_estimated_b.drop("dew_or_rime:idx", axis=1)\ndm.X_train_estimated_c = dm.X_train_estimated_c.drop("dew_or_rime:idx", axis=1)\n\ndm.X_test_estimated_a = dm.X_test_estimated_a.drop("dew_or_rime:idx", axis=1)\ndm.X_test_estimated_b = dm.X_test_estimated_b.drop("dew_or_rime:idx", axis=1)\ndm.X_test_estimated_c = dm.X_test_estimated_c.drop("dew_or_rime:idx", axis=1)\n\ndm.X_train_observed_a = dm.X_train_observed_a.drop("prob_rime:p", axis=1)\ndm.X_train_observed_b = dm.X_train_observed_b.drop("prob_rime:p", axis=1) \ndm.X_train_observed_c = dm.X_train_observed_c.drop("prob_rime:p", axis=1) \n\ndm.X_train_estimated_a = dm.X_train_estimat

### Elevation

In [299]:
dm.X_train_observed_a = dm.X_train_observed_a.drop("elevation:m", axis=1)
dm.X_train_observed_b = dm.X_train_observed_b.drop("elevation:m", axis=1) 
dm.X_train_observed_c = dm.X_train_observed_c.drop("elevation:m", axis=1) 

dm.X_train_estimated_a = dm.X_train_estimated_a.drop("elevation:m", axis=1)
dm.X_train_estimated_b = dm.X_train_estimated_b.drop("elevation:m", axis=1)
dm.X_train_estimated_c = dm.X_train_estimated_c.drop("elevation:m", axis=1)

dm.X_test_estimated_a = dm.X_test_estimated_a.drop("elevation:m", axis=1)
dm.X_test_estimated_b = dm.X_test_estimated_b.drop("elevation:m", axis=1)
dm.X_test_estimated_c = dm.X_test_estimated_c.drop("elevation:m", axis=1)

## Data editing

### Resample

In [300]:

def resample(df:pd.DataFrame): 

    categorical_feature = ["date_forecast", "is_day:idx", "is_in_shadow:idx", "clear_sky_energy_1h:J", "diffuse_rad_1h:J", "direct_rad_1h:J", "fresh_snow_12h:cm", "fresh_snow_24h:cm", "precip_type_5min:idx"]

    categorical_set = df.loc[:, categorical_feature][::4].copy()

    resampled_set = df.resample('1H', on="date_forecast").agg({"precip_5min:mm":np.sum, "rain_water:kgm2":np.sum, "snow_water:kgm2":np.sum, "super_cooled_liquid_water:kgm2":np.mean, 
                                                                "absolute_humidity_2m:gm3":np.mean, "air_density_2m:kgm3":np.mean, "clear_sky_rad:W":np.mean, "dew_point_2m:K":np.mean, "clear_sky_rad:W":np.mean, "diffuse_rad:W":np.mean, "direct_rad:W":np.mean, "effective_cloud_cover:p":np.mean, "msl_pressure:hPa":np.mean, "pressure_100m:hPa":np.mean, "pressure_50m:hPa":np.mean, "relative_humidity_1000hPa:p":np.mean, "sfc_pressure:hPa":np.mean, "sun_azimuth:d":np.mean, "sun_elevation:d":np.mean, "t_1000hPa:K":np.mean, "total_cloud_cover:p":np.mean, "visibility:m":np.mean, "wind_speed_10m:ms":np.mean, "wind_speed_u_10m:ms":np.mean, "wind_speed_v_10m:ms":np.mean, "cloud_base_agl:m":np.mean}).copy()

    combined = pd.merge(categorical_set, resampled_set, how="left", on="date_forecast")
    
    return combined

dm.X_train_observed_a = resample(dm.X_train_observed_a)
dm.X_train_observed_b = resample(dm.X_train_observed_b)
dm.X_train_observed_c = resample(dm.X_train_observed_c)

dm.X_train_estimated_a = resample(dm.X_train_estimated_a)
dm.X_train_estimated_b = resample(dm.X_train_estimated_b)
dm.X_train_estimated_c = resample(dm.X_train_estimated_c)

dm.X_test_estimated_a = resample(dm.X_test_estimated_a)
dm.X_test_estimated_b = resample(dm.X_test_estimated_b)
dm.X_test_estimated_c = resample(dm.X_test_estimated_c)


### Combining datasets

In [301]:
def combine(obs:pd.DataFrame, es:pd.DataFrame, train:pd.DataFrame): 
    weather_data = pd.concat([obs, es], axis=0)

    data = pd.merge(weather_data, train, how="left", on="date_forecast").dropna().reset_index(drop=True)

    return data

dm.data_A = combine(dm.X_train_observed_a, dm.X_train_estimated_a, dm.train_a)
dm.data_B = combine(dm.X_train_observed_b, dm.X_train_estimated_b, dm.train_b)
dm.data_C = combine(dm.X_train_observed_c, dm.X_train_estimated_c, dm.train_c)


### Constant intervals

In [302]:
#fig, axs = plt.subplots(1, 1, figsize=(20, 10))

#plt.scatter(dm.data_B["date_forecast"], dm.data_B["pv_measurement"], s=2)


In [303]:
# dm.remove_constant_periods(24)
# dm.remove_constant_periods(3, [0])

val = None
counter = 0
consequtive_zero_threshold = 23
threshold = 0.0

def remove_const_intervals(row):
    global val
    global counter
    global consequtive_zero_threshold

    if val is None: # init val as row 
        val = row
        return False

    elif val == 0.0: ## if the current value is 0.0 

        counter += 1 ## we count this 

        if counter >= consequtive_zero_threshold: ## number of seen consequtive zeros are above threshold
            val = row
            counter += 1
            return False
        
        else: 
            val = row
            return True ## we have not yet gone past threshold

    elif abs(row-val)<= threshold: ## we see a value (not zero) if below threshold we remove, this also indicates a break in any streak of zeros, setting counter to zero
        val = row
        counter = 0
        return False
    
    counter = 0
    val = row
    return True ## if all else fails, we keep the row. 


dm.data_A = dm.data_A[dm.data_A['pv_measurement'].map(remove_const_intervals)].reset_index(drop=True)
dm.data_B = dm.data_B[dm.data_B['pv_measurement'].map(remove_const_intervals)].reset_index(drop=True)
dm.data_C = dm.data_C[dm.data_C['pv_measurement'].map(remove_const_intervals)].reset_index(drop=True)


In [304]:
#fig, axs = plt.subplots(1, 1, figsize=(20, 10))

#plt.scatter(dm.data_B["date_forecast"], dm.data_B["pv_measurement"], s=2)


### B-C donation

In [305]:

# def donate_missing_pv(donating:pd.DataFrame, recieving:pd.DataFrame, scaling:float=1):
    
#     recieving_frame = recieving.copy()

#     recieving_dates = np.array(recieving_frame["date_forecast"])
#     beginning_date = recieving_dates[0]

#     for index, row in donating.iterrows():

#         if row["date_forecast"] not in recieving_dates and row["date_forecast"] > beginning_date: 

#             row = row
#             if row["pv_measurement"] == 0.0 or row["pv_measurement"] == -0.0: 
#                 pv = 0.0

#             else : 
#                 pv = row["pv_measurement"] * scaling


#             row["pv_measurement"] = pv

#             recieving_frame.loc[len(recieving_frame.index)] = row 

    
#     return recieving_frame.sort_values(by="date_forecast", ascending=True).reset_index(drop=True)

# data_B = donate_missing_pv(dm.data_C, dm.data_B, 1.2)
# data_C = donate_missing_pv(dm.data_B, dm.data_C, 0.8)


In [306]:
# dm.data_B = data_B
# dm.data_C = data_C

### Combining locations

In [307]:
dm.combine_all_data()

dm.sorting_columns_inMainSets()

['X_test_estimated_a', 'data_A']
['X_test_estimated_b', 'data_B']
['X_test_estimated_c', 'data_C']


### Data types

In [308]:
dm.data_A.dtypes 

dm.data_A[list(dm.data_A.select_dtypes(include='float32'))] = dm.data_A[list(dm.data_A.select_dtypes(include='float32'))].astype('float64')
dm.data_B[list(dm.data_B.select_dtypes(include='float32'))] = dm.data_B[list(dm.data_B.select_dtypes(include='float32'))].astype('float64')
dm.data_C[list(dm.data_C.select_dtypes(include='float32'))] = dm.data_C[list(dm.data_C.select_dtypes(include='float32'))].astype('float64')

dm.X_test_estimated_a[list(dm.X_test_estimated_a.select_dtypes(include='float32'))] = dm.X_test_estimated_a[list(dm.X_test_estimated_a.select_dtypes(include='float32'))].astype('float64')
dm.X_test_estimated_b[list(dm.X_test_estimated_b.select_dtypes(include='float32'))] = dm.X_test_estimated_b[list(dm.X_test_estimated_b.select_dtypes(include='float32'))].astype('float64')
dm.X_test_estimated_c[list(dm.X_test_estimated_c.select_dtypes(include='float32'))] = dm.X_test_estimated_c[list(dm.X_test_estimated_c.select_dtypes(include='float32'))].astype('float64')


# Feature engineering

## Part 1

### Pool data

In [309]:
# Set variables

data_A = dm.data_A
data_B = dm.data_B
data_C = dm.data_C

test_A = dm.X_test_estimated_a
test_B = dm.X_test_estimated_b
test_C = dm.X_test_estimated_c

In [310]:
# Mark estimated data

est_dates_A = pd.to_datetime(dm.X_train_estimated_a['date_forecast'])
est_dates_B = pd.to_datetime(dm.X_train_estimated_b['date_forecast'])
est_dates_C = pd.to_datetime(dm.X_train_estimated_c['date_forecast'])

data_A['est'] = (pd.to_datetime(data_A['date_forecast']).isin(est_dates_A)).astype(int)
data_B['est'] = (pd.to_datetime(data_B['date_forecast']).isin(est_dates_B)).astype(int)
data_C['est'] = (pd.to_datetime(data_C['date_forecast']).isin(est_dates_C)).astype(int)

test_A['est'] = 1
test_B['est'] = 1
test_C['est'] = 1

In [311]:
# Create full dataset

data_A['train'] = 1
data_B['train'] = 1
data_C['train'] = 1

test_A['train'] = 0
test_B['train'] = 0
test_C['train'] = 0

test_A['pv_measurement'] = np.nan
test_B['pv_measurement'] = np.nan
test_C['pv_measurement'] = np.nan

data_test_A = pd.concat([data_A, test_A], ignore_index=True)
data_test_B = pd.concat([data_B, test_B], ignore_index=True)
data_test_C = pd.concat([data_C, test_C], ignore_index=True)

In [312]:
# Set location as categorical

data_test_A['A'] = 1
data_test_A['B'] = 0
data_test_A['C'] = 0

data_test_B['A'] = 0
data_test_B['B'] = 1
data_test_B['C'] = 0

data_test_C['A'] = 0
data_test_C['B'] = 0
data_test_C['C'] = 1

data_test_A = data_test_A.drop('location', axis='columns')
data_test_B = data_test_B.drop('location', axis='columns')
data_test_C = data_test_C.drop('location', axis='columns')


### Precip type to category

In [313]:
# Allready removed


categories = ['precip_none', 'precip_rain', 'precip_rain_Snow', 'precip_snow', 'precip_sleet', 'precip_freezing_rain', 'precip_hail']

def add_precip_category(df, categories):
    # Assuming 'precip_type_5min:idx' contains integer category indices
    # Ensure the indices are integers
    df['precip_type_5min:idx'] = df['precip_type_5min:idx'].astype(int)

    # Get dummies and concatenate with the original dataframe
    dummies = pd.get_dummies(df['precip_type_5min:idx'], prefix='precip_type').astype(int)
    df = pd.concat([df, dummies], axis=1)

    return df

# Apply the function
data_test_A = add_precip_category(data_test_A, categories)
data_test_B = add_precip_category(data_test_B, categories)
data_test_C = add_precip_category(data_test_C, categories)
# Drop precip_type
data_test_A = data_test_A.drop('precip_type_5min:idx', axis='columns')
data_test_B = data_test_B.drop('precip_type_5min:idx', axis='columns')
data_test_C = data_test_C.drop('precip_type_5min:idx', axis='columns')


### Temporal features

In [314]:
# Dates columns

def extract_dates(df):
    # Convert 'date_forecast' to datetime
    df['date_forecast'] = pd.to_datetime(df['date_forecast'])

    # Extract year, month, and day
    df['year'] = df['date_forecast'].dt.year
    df['month'] = df['date_forecast'].dt.month
    df['day'] = df['date_forecast'].dt.day

    return df

data_test_A = extract_dates(data_test_A)
data_test_B = extract_dates(data_test_B)
data_test_C = extract_dates(data_test_C)


In [315]:
# Add daily and annual sinus curve

def hour_func(x): 
    return np.cos(2*np.pi/24 * x)

data_test_A["daily_sinus"] = hour_func(data_test_A["date_forecast"].dt.hour)
data_test_B["daily_sinus"] = hour_func(data_test_B["date_forecast"].dt.hour)
data_test_C["daily_sinus"] = hour_func(data_test_C["date_forecast"].dt.hour)

#plot_feature(data_test_A.iloc[:40], 'daily_sinus')

def day_func(x): 
    return np.cos(2*np.pi/365 * x - 2*np.pi/365 * 173)

data_test_A["annual_sinus"] = day_func(data_test_A["date_forecast"].dt.dayofyear)
data_test_B["annual_sinus"] = day_func(data_test_B["date_forecast"].dt.dayofyear)
data_test_C["annual_sinus"] = day_func(data_test_C["date_forecast"].dt.dayofyear)

#plot_feature(data_test_A.iloc[8000:10000], 'annual_sinus')

### Mark weird cloud data as category

In [316]:
data_test_A['bad_cloud_data'] = (data_test_A['date_forecast'] < pd.to_datetime('2020-03-26 00:00:00')).astype(int)
data_test_B['bad_cloud_data'] = (data_test_B['date_forecast'] < pd.to_datetime('2020-03-26 00:00:00')).astype(int)
data_test_C['bad_cloud_data'] = (data_test_C['date_forecast'] < pd.to_datetime('2020-03-26 00:00:00')).astype(int)

### Clip sun elevation

In [317]:
# When sun elevation is below ca. -5, no pv_measurement is recorded

data_test_A['sun_elevation:d'] = data_test_A['sun_elevation:d'].clip(lower=0)
data_test_B['sun_elevation:d'] = data_test_B['sun_elevation:d'].clip(lower=0)
data_test_C['sun_elevation:d'] = data_test_C['sun_elevation:d'].clip(lower=0)

### Cloud data

In [318]:
# Create mask for open sky, clip negative values for clouds

data_test_A['open_sky'] = ( data_test_A['cloud_base_agl:m'] < 0).astype(int)
data_test_B['open_sky'] = ( data_test_B['cloud_base_agl:m'] < 0).astype(int)
data_test_C['open_sky'] = ( data_test_C['cloud_base_agl:m'] < 0).astype(int)

#data_test_A['cloud_base_agl:m'] = data_test_A['cloud_base_agl:m'].clip(lower=0)
#data_test_B['cloud_base_agl:m'] = data_test_B['cloud_base_agl:m'].clip(lower=0)
#data_test_C['cloud_base_agl:m'] = data_test_C['cloud_base_agl:m'].clip(lower=0)


### Lag and lead

In [319]:
# Create lag and lead for direct sun rad

def create_lag_average_column(df, column_name, lag_window):
    """
    Creates a new column in the DataFrame which is the average of the last 3 rows of the specified column.
    For the first two rows, it uses the available values for averaging.

    :param df: Pandas DataFrame.
    :param column_name: The name of the column for which the lagged average is calculated.
    :return: DataFrame with the new lag average column added.
    """
    # Ensure that the DataFrame has the specified column
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Calculate the rolling average of the last 3 rows
    df[f'{column_name}_lag_avg'] = df[column_name].rolling(window=lag_window, min_periods=1).mean()

    return df

lag_window = 10

data_test_A = create_lag_average_column(data_test_A, 'direct_rad:W', lag_window)
data_test_B = create_lag_average_column(data_test_B, 'direct_rad:W', lag_window)
data_test_C = create_lag_average_column(data_test_C, 'direct_rad:W', lag_window)

#data_test_A = create_lag_average_column(data_test_A, 'diffuse_rad:W', lag_window)
#data_test_B = create_lag_average_column(data_test_B, 'diffuse_rad:W', lag_window)
#data_test_C = create_lag_average_column(data_test_C, 'diffuse_rad:W', lag_window)


In [320]:
# Continuing with lead

def create_lead_average_column(df, column_name, lead_window):
    """
    Creates a new column in the DataFrame which is the average of the next 3 rows of the specified column.
    For the last two rows, it uses the available values for averaging.

    :param df: Pandas DataFrame.
    :param column_name: The name of the column for which the lead average is calculated.
    :return: DataFrame with the new lead average column added.
    """
    # Ensure that the DataFrame has the specified column
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Reverse the DataFrame, calculate the rolling average, and then reverse it back
    df_reversed = df.iloc[::-1].copy()
    df_reversed[f'{column_name}_lead_avg'] = df_reversed[column_name].rolling(window=lead_window, min_periods=1).mean()
    df[f'{column_name}_lead_avg'] = df_reversed[f'{column_name}_lead_avg'].iloc[::-1].values

    return df

lead_window = 1

data_test_A = create_lead_average_column(data_test_A, 'direct_rad:W', lead_window)
data_test_B = create_lead_average_column(data_test_B, 'direct_rad:W', lead_window)
data_test_C = create_lead_average_column(data_test_C, 'direct_rad:W', lead_window)

#data_test_A = create_lead_average_column(data_test_A, 'diffuse_rad:W', lead_window)
#data_test_B = create_lead_average_column(data_test_B, 'diffuse_rad:W', lead_window)
#data_test_C = create_lead_average_column(data_test_C, 'diffuse_rad:W', lead_window)

In [321]:
# Create central derivative column

def create_central_difference_column(df, column_name, new_column_name):
    """
    Creates a new column in the DataFrame which is the central difference of the specified column with respect
    to the previous and next row. For the first and last rows, forward and backward differences are used.

    :param df: Pandas DataFrame.
    :param column_name: The name of the column for which the central difference is calculated.
    :param new_column_name: The name of the new column to store the central difference.
    :return: DataFrame with the new central difference column added.
    """

    # Ensure that the DataFrame has the specified column
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Calculate the central difference
    df[new_column_name] = (df[column_name].shift(-1) - df[column_name].shift(1)) / 2

    # Handle the first row using forward difference
    df.at[0, new_column_name] = df.at[1, column_name] - df.at[0, column_name]

    # Handle the last row using backward difference
    last_idx = df.index[-1]
    df.at[last_idx, new_column_name] = df.at[last_idx, column_name] - df.at[last_idx - 1, column_name]

    return df

data_test_A = create_central_difference_column(data_test_A, 'direct_rad:W', 'direct_rad_CD')
data_test_B = create_central_difference_column(data_test_B, 'direct_rad:W', 'direct_rad_CD')
data_test_C = create_central_difference_column(data_test_C, 'direct_rad:W', 'direct_rad_CD')

### Difference between columns

In [322]:
# Create difference between columns

def create_difference_column(df, column1, column2, new_column_name):
    """
    Creates a new column in the DataFrame which is the difference between two specified columns.

    :param df: Pandas DataFrame.
    :param column1: The name of the first column.
    :param column2: The name of the second column.
    :param new_column_name: The name of the new column to store the difference.
    :return: DataFrame with the new difference column added.
    """
    # Ensure that the DataFrame has the specified columns
    if column1 not in df.columns:
        raise ValueError(f"Column '{column1}' not found in the DataFrame.")
    if column2 not in df.columns:
        raise ValueError(f"Column '{column2}' not found in the DataFrame.")

    # Calculate the difference and store it in the new column
    df[new_column_name] = df[column1] - df[column2]

    return df


#data_test_A = create_difference_column(data_test_A, 'is_day:idx', 'is_in_shadow:idx', 'diff_day_shadow')


### Drop useless columns

In [323]:
# Drop useless columns
useless_col = [
    'fresh_snow_12h:cm',
    'fresh_snow_24h:cm',
    'pressure_50m:hPa',
    'msl_pressure:hPa',
    'sfc_pressure:hPa',
    'precip_type_2',
    'precip_type_3',
]

data_test_A = data_test_A.drop(useless_col, axis='columns')
data_test_B = data_test_B.drop(useless_col, axis='columns')
data_test_C = data_test_C.drop(useless_col, axis='columns')

data_test_A = data_test_A.drop(['precip_type_5', 'precip_type_6'], axis='columns')


### Split, combine, sort

In [324]:
# Split by train / test

data_A = data_test_A[ data_test_A['train'] == 1 ]
data_B = data_test_B[ data_test_B['train'] == 1 ]
data_C = data_test_C[ data_test_C['train'] == 1 ]

test_A = data_test_A[ data_test_A['train'] == 0 ]
test_B = data_test_B[ data_test_B['train'] == 0 ]
test_C = data_test_C[ data_test_C['train'] == 0 ]


In [325]:
# Drop NaN pv_measurement
test_A = test_A.drop('pv_measurement', axis='columns')
test_B = test_B.drop('pv_measurement', axis='columns')
test_C = test_C.drop('pv_measurement', axis='columns')

In [326]:
# Combine
data_ALL = pd.concat([data_A, data_B, data_C], ignore_index=True)
test_ALL = pd.concat([test_A, test_B, test_C], ignore_index=True)

In [327]:
# Sort by date forecast
data_ALL = data_ALL.sort_values(['date_forecast', 'A', 'B', 'C'], ascending=[True, False, False, False])
test_ALL = test_ALL.sort_values(['date_forecast', 'A', 'B', 'C'], ascending=[True, False, False, False])

In [328]:
# Rename
data_ALL.rename(columns={'pv_measurement': 'target'}, inplace=True)

## Part 2

### Set X and y

In [329]:
# Separate features and target variable
X = data_ALL.drop('target', axis='columns')
y = data_ALL[['date_forecast', 'target']]
X_kaggle = test_ALL

In [330]:
# Split data
def split_data(df, percent):
    split_index = int( np.floor( len(df)*percent ) )
    df_first = df[:split_index]
    df_last = df[split_index:]
    return df_first, df_last

train_percent = 0.92 # Of all
val_percent = 0.5 # Of non-train

X_train, X_non_train = split_data(X, train_percent)
X_val, X_test = split_data(X_non_train, val_percent)

y_train, y_non_train = split_data(y, train_percent)
y_val, y_test = split_data(y_non_train, val_percent)

### Set val data to summer last year (optional)

In [331]:
"""
X['date_forecast'] = pd.to_datetime(X['date_forecast'])
y['date_forecast'] = pd.to_datetime(y['date_forecast'])

# Define your date range
start_date = "2022-04-01 00:00:00"
end_date = "2022-08-03 23:00:00"

# Convert strings to datetime
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Filter rows within the date range
mask_X = (X['date_forecast'] >= start_date) & (X['date_forecast'] <= end_date)
mask_y = (y['date_forecast'] >= start_date) & (y['date_forecast'] <= end_date)
X_val = X.loc[mask_X]
y_val = y.loc[mask_y]

# Pop out rows within the date range to remove them from the original df
X_train = X.loc[~mask_X]
y_train = y.loc[~mask_y]
"""

'\nX[\'date_forecast\'] = pd.to_datetime(X[\'date_forecast\'])\ny[\'date_forecast\'] = pd.to_datetime(y[\'date_forecast\'])\n\n# Define your date range\nstart_date = "2022-04-01 00:00:00"\nend_date = "2022-08-03 23:00:00"\n\n# Convert strings to datetime\nstart_date = pd.to_datetime(start_date)\nend_date = pd.to_datetime(end_date)\n\n# Filter rows within the date range\nmask_X = (X[\'date_forecast\'] >= start_date) & (X[\'date_forecast\'] <= end_date)\nmask_y = (y[\'date_forecast\'] >= start_date) & (y[\'date_forecast\'] <= end_date)\nX_val = X.loc[mask_X]\ny_val = y.loc[mask_y]\n\n# Pop out rows within the date range to remove them from the original df\nX_train = X.loc[~mask_X]\ny_train = y.loc[~mask_y]\n'

### Filter X and y by summer months

In [332]:
#y = y[ X['date_forecast'].dt.month.between(4, 8) ]
#X = X[ X['date_forecast'].dt.month.between(4, 8) ]

### Load external data (optional)

In [333]:
test_ALL_to_sub = pd.read_csv("current_csv_files/test_ALL.csv")
test_ALL_to_sub_ABC = test_ALL_to_sub.sort_values(['A', 'B', 'C', 'date_forecast'], ascending=[False, False, False, True])
y_hat_kaggle = pd.read_csv("teo_subs/kaggle_149.csv", index_col='id')
test_ALL_to_sub_ABC['new_index'] = range(2160)
test_ALL_to_sub_ABC = test_ALL_to_sub_ABC.set_index('new_index')
test_ALL_to_sub_ABC['y_hat_kaggle'] = y_hat_kaggle
test_ALL_to_sub_sorted = test_ALL_to_sub_ABC.sort_values(['date_forecast', 'A', 'B', 'C'], ascending=[True, False, False, False])
test_ALL_to_sub_sorted = test_ALL_to_sub_sorted.set_index('date_forecast')
y_hat = test_ALL_to_sub_sorted['y_hat_kaggle']

### Set date as index

In [334]:
X = X.set_index('date_forecast')
y = y.set_index('date_forecast')

X_train = X_train.set_index('date_forecast')
y_train = y_train.set_index('date_forecast')

X_non_train = X_non_train.set_index('date_forecast')
y_non_train = y_non_train.set_index('date_forecast')

X_val = X_val.set_index('date_forecast')
y_val = y_val.set_index('date_forecast')

X_test = X_test.set_index('date_forecast')
y_test = y_test.set_index('date_forecast')

X_kaggle = X_kaggle.set_index('date_forecast')

### Feature scaling

In [335]:
# Prepare features for scaling

feature_dont_touch = [
    'date_forecast',
    'is_day:idx',
    'is_in_shadow:idx',
    'pv_measurement',
    'est',
    'train',
    'A',
    'B',
    'C',
    'daily_sinus',
    'annual_sinus',
    'bad_cloud_data',
    'open_sky',

    # Kinda useless
    'dew_or_rime:idx'
]

feature_to_standardize = [
    'absolute_humidity_2m:gm3',
    'air_density_2m:kgm3',
    'dew_point_2m:K',
    'pressure_100m:hPa',
    'relative_humidity_1000hPa:p',
    't_1000hPa:K',
    'wind_speed_u_10m:ms',
    'wind_speed_v_10m:ms',
    'direct_rad_CD', # Central difference

    # Kinda useless
    #'pressure_50m:hPa',
    #'msl_pressure:hPa',
    #'sfc_pressure:hPa',
]

feature_to_normalize = [
    'cloud_base_agl:m',
    'clear_sky_energy_1h:J',
    'diffuse_rad_1h:J',
    'direct_rad_1h:J',
    'precip_5min:mm',
    'rain_water:kgm2',
    'snow_water:kgm2',
    'super_cooled_liquid_water:kgm2',
    'clear_sky_rad:W',
    'diffuse_rad:W',
    'direct_rad:W',
    'direct_rad:W_lag_avg', # Lag
    'direct_rad:W_lead_avg', # Lead
    'effective_cloud_cover:p',
    'sun_azimuth:d',
    'total_cloud_cover:p',
    'visibility:m',
    'wind_speed_10m:ms',
    'sun_elevation:d', # Clipped version
    'year',
    'month',
    'day',

    # Kinda useless
    #'fresh_snow_12h:cm',
    #'fresh_snow_24h:cm'
]

In [336]:
# Scale features

standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

for feature in feature_to_standardize:
    #X_train[feature] = standard_scaler.fit_transform(X_train[[feature]])
    X[feature] = standard_scaler.fit_transform(X[[feature]])
    
    X_non_train[feature] = standard_scaler.transform(X_non_train[[feature]])
    X_val[feature] = standard_scaler.transform(X_val[[feature]])
    X_test[feature] = standard_scaler.transform(X_test[[feature]])
    X_kaggle[feature] = standard_scaler.transform(X_kaggle[[feature]])
    

for feature in feature_to_normalize:
    #X_train[feature] = standard_scaler.fit_transform(X_train[[feature]])
    X[feature] = min_max_scaler.fit_transform(X[[feature]])
    
    X_non_train[feature] = min_max_scaler.transform(X_non_train[[feature]])
    X_val[feature] = min_max_scaler.transform(X_val[[feature]])
    X_test[feature] = min_max_scaler.transform(X_test[[feature]])
    X_kaggle[feature] = min_max_scaler.transform(X_kaggle[[feature]])

### Split by location

In [337]:
"""
X_A = X[ X['A'] == 1 ]
X_B = X[ X['B'] == 1 ]
X_C = X[ X['C'] == 1 ]

y_A = y[ X['A'] == 1 ]
y_B = y[ X['B'] == 1 ]
y_C = y[ X['C'] == 1 ]

X_A_kaggle = X_kaggle[ X_kaggle['A'] == 1 ]
X_B_kaggle = X_kaggle[ X_kaggle['B'] == 1 ]
X_C_kaggle = X_kaggle[ X_kaggle['C'] == 1 ]

y_A_hat = y_hat[ X_kaggle['A'] == 1 ]
y_B_hat = y_hat[ X_kaggle['B'] == 1 ]
y_C_hat = y_hat[ X_kaggle['C'] == 1 ]
"""

"\nX_A = X[ X['A'] == 1 ]\nX_B = X[ X['B'] == 1 ]\nX_C = X[ X['C'] == 1 ]\n\ny_A = y[ X['A'] == 1 ]\ny_B = y[ X['B'] == 1 ]\ny_C = y[ X['C'] == 1 ]\n\nX_A_kaggle = X_kaggle[ X_kaggle['A'] == 1 ]\nX_B_kaggle = X_kaggle[ X_kaggle['B'] == 1 ]\nX_C_kaggle = X_kaggle[ X_kaggle['C'] == 1 ]\n\ny_A_hat = y_hat[ X_kaggle['A'] == 1 ]\ny_B_hat = y_hat[ X_kaggle['B'] == 1 ]\ny_C_hat = y_hat[ X_kaggle['C'] == 1 ]\n"

# ML models

## Deep neural network

In [338]:
class CustomModelCheckpoint(Callback):
    def __init__(self, best_weights, best_epochs, i):
        self.best_weights = best_weights  # Dictionary to store best weights
        self.best_epochs = best_epochs    # Dictionary to store best epoch numbers
        self.best_loss = np.inf
        self.model_index = i  # Index to handle the specific model in the list

    def on_epoch_end(self, epoch, logs=None):
        # Monitor the validation loss
        current_val_loss = logs.get('val_loss')
        if current_val_loss < self.best_loss:
            # Update the best loss, best weights, and best epoch
            self.best_loss = current_val_loss
            self.best_weights[self.model_index] = self.model.get_weights()
            self.best_epochs[self.model_index] = epoch
            

In [345]:
def objective(trial):

    
    trial_params = {
        'seed': trial.number + 50,
        'N': trial.suggest_int('N', 1, 3),
        'n_neurons_1': trial.suggest_int('n_neurons_1', 140, 180),
        'n_neurons_2': trial.suggest_int('n_neurons_2', 150, 200),
        'n_neurons_3': trial.suggest_int('n_neurons_3', 150, 200),
        'n_neurons_3.5': trial.suggest_int('n_neurons_3', 90, 130),
        'n_neurons_4': trial.suggest_int("n_neurons_4", 60, 90),
        'Dropout': trial.suggest_float('Dropout', 0.05, 0.2),
        'kernel_regularizer': trial.suggest_float('kernel_regularizer', 0.05, 0.25),
        'learning_rate': trial.suggest_float('learning_rate', 0.00075, 0.0012),
        'beta_1': trial.suggest_float('beta_1', 0.75, 0.9),
        'min_delta': trial.suggest_float('min_delta', 5, 20),
        'patience': 10, #trial.suggest_int('patience', 3, 8),
        'batch_size': trial.suggest_categorical("batch_size", [16, 32, 64])
    }

    s = trial_params['seed']
    print("Seed: ", str(s))
    np.random.seed(s)
    random.seed(s)
    tf.random.set_seed(s)

    N = trial_params['N']
    nets = []
    init = 'HeNormal'

    # Initialize dictionary to store the best weights for each model
    best_weights_during_training = {}
    best_epochs_during_training = {}

    for i in range(N):
        # Define the Keras model
        model = Sequential([
            Dense(trial_params["n_neurons_1"], input_dim=X.shape[1], activation='relu', kernel_initializer=init),
            Dropout(trial_params['Dropout']),

            Dense(trial_params["n_neurons_2"], activation='relu', kernel_initializer=init, kernel_regularizer=l2(trial_params["kernel_regularizer"])),
            Dropout(trial_params['Dropout']),

            Dense(trial_params["n_neurons_3"], activation='relu', kernel_initializer=init, kernel_regularizer=l2(trial_params["kernel_regularizer"])),
            Dropout(trial_params['Dropout']),

            Dense(trial_params["n_neurons_3.5"], activation='relu', kernel_initializer=init, kernel_regularizer=l2(trial_params["kernel_regularizer"])),
            Dropout(trial_params['Dropout']),
            
            Dense(trial_params["n_neurons_4"], activation='relu', kernel_initializer=init, kernel_regularizer=l2(trial_params["kernel_regularizer"])),
            Dense(1, kernel_initializer=init)
        ])
        nets.append(model)

        adam = Adam(learning_rate=trial_params["learning_rate"], beta_1=trial_params["beta_1"])
        model.compile(loss='mean_absolute_error', optimizer=adam)

        # Define early stopping
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=trial_params['min_delta'], patience=trial_params['patience'])

        # Create an instance of the custom checkpoint for the current model
        custom_checkpoint = CustomModelCheckpoint(best_weights_during_training, best_epochs_during_training, i)
        
        # Fit the model
        history = model.fit(
            X, y,
            validation_data=(X_kaggle, y_hat),
            #validation_split=0.2,
            epochs=100,
            batch_size=trial_params['batch_size'],
            callbacks=[early_stopping, custom_checkpoint],
            verbose=0,
            use_multiprocessing=True, workers=4,
        )


        if i not in best_weights_during_training:
            # If the model didn't improve, just save the last weights and epoch number
            best_weights_during_training[i] = model.get_weights()
            best_epochs_during_training[i] = len(history.epoch)


    # Set weights and print epoch
    for i in range(N):
        nets[i].set_weights(best_weights_during_training[i])
        print("Best epoch: ", best_epochs_during_training[i] +1) # It's plus one for some reason. Idk it works
    
    # Make kaggle prediction
    best_kaggle_preds = []

    for i in range(N):
        best_kaggle_preds.append(nets[i].predict(X_kaggle, verbose=0).ravel())
    
    # Create dataframe for kaggle pred
    df_kaggle_preds = pd.DataFrame(best_kaggle_preds).T
    df_kaggle_preds['avg'] = df_kaggle_preds.mean(axis='columns')

    # Merge with kaggle_test data
    test_ALL_merge = pd.read_csv("current_csv_files/test_ALL.csv")
    test_ALL_merge['prediction'] = df_kaggle_preds['avg']
    

    # Correctly sort test data for submission
    test_ALL_merge_sorted = test_ALL_merge.sort_values(['A', 'B', 'C', 'date_forecast'], ascending=[False, False, False, True])
    test_ALL_merge_sorted['id'] = range(2160)
    test_ALL_merge_sorted = test_ALL_merge_sorted.set_index('id')
    test_ALL_merge_sorted['id'] = range(2160)

    # Comparison to best sub on kaggle
    #best_sub = pd.read_csv("teo_subs/best_sub.csv", index_col='id')
    objective_metric = mean_absolute_error(df_kaggle_preds['avg'], y_hat)
    print(trial.number," MAE: ", mean_absolute_error(df_kaggle_preds['avg'], y_hat))
    print(trial.number," RMSE: ", np.sqrt(mean_squared_error(df_kaggle_preds['avg'], y_hat)))

    file_loc = "dnn_kaggle3/optuna_sub_"+str(trial.number)+".csv"
    test_ALL_merge_sorted[['id', 'prediction']].to_csv(file_loc, index=False)

    return objective_metric
    

In [346]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

print('Best hyperparameters:', study.best_params)
print('Best MSE:', study.best_value)

[I 2023-11-11 21:49:12,165] A new study created in memory with name: no-name-d8bcafcb-1617-4ea3-9973-056a8e943b01
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/optuna/trial/_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "n_neurons_3"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 150, 'high': 200}
  warnings.warn(


Seed:  50
Best epoch:  2
Best epoch:  5
Best epoch:  4


[I 2023-11-11 21:52:46,170] Trial 0 finished with value: 58.35341188195377 and parameters: {'N': 3, 'n_neurons_1': 143, 'n_neurons_2': 184, 'n_neurons_3': 189, 'n_neurons_4': 62, 'Dropout': 0.06698146903599685, 'kernel_regularizer': 0.1895926421254273, 'learning_rate': 0.000985184837191271, 'beta_1': 0.8943031675139399, 'min_delta': 9.162014886489292, 'batch_size': 16}. Best is trial 0 with value: 58.35341188195377.


0  MAE:  58.35341188195377
0  RMSE:  112.47948015873651
Seed:  51


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/optuna/trial/_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "n_neurons_3"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 150, 'high': 200}
  warnings.warn(
[I 2023-11-11 21:53:20,410] Trial 1 finished with value: 63.76180585626043 and parameters: {'N': 1, 'n_neurons_1': 140, 'n_neurons_2': 180, 'n_neurons_3': 156, 'n_neurons_4': 64, 'Dropout': 0.08109373800033151, 'kernel_regularizer': 0.20744873692984894, 'learning_rate': 0.001087608185587972, 'beta_1': 0.8444271552561563, 'min_delta': 19.922290826524897, 'batch_size': 32}. Best is trial 0 with value: 58.35341188195377.


...

Best epoch:  14
Best epoch:  7


[I 2023-11-12 14:26:39,710] Trial 13 finished with value: 59.577933646059535 and parameters: {'N': 2, 'n_neurons_1': 149, 'n_neurons_2': 172, 'n_neurons_3': 193, 'n_neurons_4': 83, 'Dropout': 0.120843358528433, 'kernel_regularizer': 0.17234205772846012, 'learning_rate': 0.001010911252155597, 'beta_1': 0.8796547711153324, 'min_delta': 5.106453050509141, 'batch_size': 16}. Best is trial 0 with value: 58.35341188195377.


13  MAE:  59.577933646059535
13  RMSE:  111.14712158324281
Seed:  64


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/optuna/trial/_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "n_neurons_3"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 150, 'high': 200}
  warnings.warn(
[W 2023-11-12 14:31:11,782] Trial 14 failed with parameters: {'N': 3, 'n_neurons_1': 155, 'n_neurons_2': 159, 'n_neurons_3': 181, 'n_neurons_4': 76, 'Dropout': 0.1550401106447224, 'kernel_regularizer': 0.12592333612327503, 'learning_rate': 0.0007663612949711176, 'beta_1': 0.8862247051819403, 'min_delta': 7.3592644981336655, 'batch_size': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.fram

KeyboardInterrupt: 

## Catboost